In [2]:
import pandas as pd

# Replace 'path_to_your_file.csv' with the actual dataset file path
df = pd.read_csv("Dataset2.csv")

# Display first five rows
df.head()


,No.,Time,Source,Destination,Protocol,Length,Info
0,10,20:09:07.212252400,192.168.0.19,192.168.0.12,HTTP2,132,HEADERS[207959]
1,11,20:09:07.212252500,192.168.0.19,192.168.0.12,HTTP2,147,DATA[207959]
2,12,20:09:07.212750800,192.168.0.12,192.168.0.19,TCP,66,7777 > 54126 [ACK] Seq=1 Ack=148 Win=501 Len...
3,13,20:09:07.212750800,192.168.0.12,192.168.0.19,HTTP2,101,HEADERS[207959]: 204 No Content
4,14,20:09:07.212948900,192.168.0.19,192.168.0.12,TCP,66,54126 > 7777 [ACK] Seq=148 Ack=36 Win=501 Le...


In [3]:
# Get dataset summary
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52185 entries, 0 to 52184
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   No.          52185 non-null  int64 
 1   Time         52185 non-null  object
 2   Source       52185 non-null  object
 3   Destination  52185 non-null  object
 4   Protocol     52185 non-null  object
 5   Length       52185 non-null  int64 
 6   Info         52185 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.8+ MB


In [4]:
# Get statistical summary
df.describe()


,No.,Length
count,52185.000000,52185.000000
mean,33722.726435,180.026540
std,18968.770167,476.105341
min,10.000000,54.000000
25%,17414.000000,59.000000
50%,34130.000000,67.000000
75%,50223.000000,110.000000
max,65842.000000,6806.000000


In [5]:
# Count missing values per column
df.isnull().sum()


No.            0
Time           0
Source         0
Destination    0
Protocol       0
Length         0
Info           0
dtype: int64

In [6]:
# Convert time to datetime format
df['Time'] = pd.to_datetime(df['Time'])

# Calculate packet rate (packets per second)
df['Packet_Rate'] = df.groupby(df['Time'].dt.second)['Length'].transform('count')

# Calculate throughput (sum of packet sizes per second)
df['Throughput'] = df.groupby(df['Time'].dt.second)['Length'].transform('sum')

# Define congestion threshold (adjust based on dataset characteristics)
packet_threshold = df['Packet_Rate'].quantile(0.75)  # Upper quartile
throughput_threshold = df['Throughput'].quantile(0.75)

# Create congestion label (1 = Congested, 0 = Normal)
df['Congestion'] = ((df['Packet_Rate'] > packet_threshold) & 
                    (df['Throughput'] > throughput_threshold)).astype(int)

# Display updated dataset
df.head()


C:\Users\Shiva\AppData\Local\Temp\ipykernel_26772\1085215439.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'])


,No.,Time,Source,Destination,Protocol,Length,Info,Packet_Rate,Throughput,Congestion
0,10,2025-02-03 20:09:07.212252,192.168.0.19,192.168.0.12,HTTP2,132,HEADERS[207959],1264,352385,1
1,11,2025-02-03 20:09:07.212252,192.168.0.19,192.168.0.12,HTTP2,147,DATA[207959],1264,352385,1
2,12,2025-02-03 20:09:07.212750,192.168.0.12,192.168.0.19,TCP,66,7777 > 54126 [ACK] Seq=1 Ack=148 Win=501 Len...,1264,352385,1
3,13,2025-02-03 20:09:07.212750,192.168.0.12,192.168.0.19,HTTP2,101,HEADERS[207959]: 204 No Content,1264,352385,1
4,14,2025-02-03 20:09:07.212948,192.168.0.19,192.168.0.12,TCP,66,54126 > 7777 [ACK] Seq=148 Ack=36 Win=501 Le...,1264,352385,1


In [8]:
from sklearn.model_selection import train_test_split

# Selecting features and target variable
X = df[['Packet_Rate', 'Throughput', 'Length']]  # Input features
y = df['Congestion']  # Target variable

# Splitting into training (80%) and testing (20%) datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display dataset sizes
X_train.shape, X_test.shape


((41748, 3), (10437, 3))

In [10]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Model training complete
print("Model Training Completed ✅")


Model Training Completed ✅


In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Make predictions on test data
y_pred = rf_model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Display classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Display confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Model Accuracy: 100.00%

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8086
           1       1.00      1.00      1.00      2351

    accuracy                           1.00     10437
   macro avg       1.00      1.00      1.00     10437
weighted avg       1.00      1.00      1.00     10437


Confusion Matrix:
 [[8086    0]
 [   0 2351]]


In [13]:
import pickle
from sklearn.ensemble import RandomForestClassifier

# Train the model again (if not already trained)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Save the trained model to a file
with open("congestion_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)

print("Model saved successfully as congestion_model.pkl ✅")


Model saved successfully as congestion_model.pkl ✅


In [14]:
!pip install matplotlib pandas



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
